In [1]:
import pandas as pd

df = pd.read_csv("../links.csv")

df[0:10]

,Unnamed: 0,category_id,content,count,id,title,url,verified_category_id
0,0,2,\n,1,1272,fake news google - Recherche Google,https://www.google.fr/search?client=firefox-b&...,NaN
1,1,2,\n,1,1271,fake news google - Recherche Google,https://www.google.fr/search?client=firefox-b&...,NaN
2,2,1,O processo de germinação do Instituto Gênesis ...,1,1270,Notícias | Instituto Gênesis,http://www.genesis.puc-rio.br/noticias/detalhe...,NaN
3,3,6,"Pediu uma pizza, ganhou a segunda! 😍😍😍 Simples...",1,1269,"Pediu uma pizza, ganhou a segunda! 😍😍😍",https://www.facebook.com/artezianforneria/phot...,NaN
4,4,1,\n,1,1268,"Testing Microservices, the sane way – Cindy Sr...",https://t.co/M77kp8eRUn,NaN
5,5,4,O Estado não gosta de concorrência ;),1,1267,O Estado não gosta de concorrência ;),https://www.facebook.com/rankingdospoliticos/p...,NaN
6,6,6,Só entre os dois? Se eu tiver que escolher só ...,1,1266,Só entre os dois? Se eu tiver que escolher só ...,https://www.facebook.com/neto.joaobatista/post...,NaN
7,7,1,"Bitcoin's price dipped below $8,000 for the fi...",1,1265,"Bitcoin price drops below $8,000 for first tim...",https://www.cnbc.com/2018/02/02/bitcoin-price-...,NaN
8,8,1,Over $100 billion was wiped off the global cry...,1,1264,Over $100 billion wiped off global cryptocurre...,https://www.cnbc.com/2018/02/02/bitcoin-ethere...,NaN
9,9,5,A nova moda entre as celebridades e famosas no...,1,1263,Mulher é proibida pela Justiça de curtir o ver...,http://www.sensacionalista.com.br/2018/02/02/m...,NaN


In [40]:
from imblearn.under_sampling import RandomUnderSampler

def undersample_data(X, y):
    columns = X.columns.values.tolist()

    X, y = RandomUnderSampler(random_state=123, ratio='all').fit_sample(
        X.values.tolist(), y.values.tolist())
    X = pd.DataFrame(X, columns=columns)

    return X, y

In [145]:
df["is_fake_news"] = [
    category_id == 2
    for category_id in df["category_id"]
]

df.dropna(subset=["title", "content"], inplace=True, how="all")

X = df[["title", "content"]]
y = df["is_fake_news"]

X, y = undersample_data(X, y)

X["title_content"] = X["title"] + X["content"]
X = X["title_content"].values.tolist()

X[0]

'"Pela primeira vez na história as pessoas têm uma chance real de tirar o poder dos governos" -https://goo.gl/ffHCgR Sei que é fake, mas achei interessante, o que acham?"Pela primeira vez na história as pessoas têm uma chance real de tirar o poder dos governos" -https://goo.gl/ffHCgR Sei que é fake, mas achei interessante, o que acham?'

In [254]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from imblearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(strip_accents='ascii', lowercase=True, ngram_range=(1, 3),
                max_df=0.5,
                min_df=5,
                use_idf=True)),
    ('sampling', RandomUnderSampler(random_state=123)),
    ('clf', MultinomialNB(fit_prior=False)),
])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

model = pipeline.fit(X_train, y_train)
accuracy = model.score(X_test, y_test)

accuracy

0.60526315789473684

In [277]:
idx = 6

print(X_test[idx])
print(y_test[idx])

model.predict_proba([X_test[idx]])

Um escândalo sem precedentes.  Marcos Feliciano e combina com Jair Bolsonaro como ajudar Eduardo Cunha a entrar com pedido de impeachment sem correr risco de perderem o processo. No mesmo áudio Feliciano,admite que estaria com um grupo, blindando e salvando a pele de Eduardo Cunha, para que ele ajudasse a derrubar Dilma Rousseff.Um escândalo sem precedentes.  Marcos Feliciano e combina com Jair Bolsonaro como ajudar Eduardo Cunha a entrar com pedido de impeachment sem correr risco de perderem o processo. No mesmo áudio Feliciano,admite que estaria com um grupo, blindando e salvando a pele de Eduardo Cunha, para que ele ajudasse a derrubar Dilma Rousseff.
True


array([[ 0.58084212,  0.41915788]])

In [278]:
from lime.lime_text import LimeTextExplainer

class_names = ["normal", "fake news"]
explainer = LimeTextExplainer(class_names=class_names)

In [283]:
exp = explainer.explain_instance(X_test[idx], model.predict_proba, num_features=30)
print('Document id: %d' % idx)
print('Probability(fake news) =', model.predict_proba([X_test[idx]])[0,1])
print('True class: %s' % class_names[y_test[idx]])

/Users/rchaves/tensorflow/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


Document id: 6
Probability(fake news) = 0.419157883147
True class: fake news


In [284]:
exp.as_list()

[('sem', 0.024867194471137232),
 ('como', -0.019653012600938331),
 ('que', -0.017929465153204423),
 ('No', -0.017336038187501377),
 ('mesmo', 0.013022728114528799),
 ('com', -0.010087556925334775),
 ('processo', 0.0089030479260908298),
 ('Um', -0.0083790154669582447),
 ('para', -0.0081072432154372237),
 ('um', -0.0080822895560579266),
 ('ele', -0.0035443795769737095),
 ('de', -0.0010398142200823059),
 ('estaria', -0.00041455606145746055),
 ('Jair', 0.00030543412095594479),
 ('impeachment', 0.00028581149299233946),
 ('Eduardo', -0.00027188807352511186),
 ('pele', 0.00026120940993973064),
 ('Feliciano', -0.00019985121480864821),
 ('a', 0.00019081468385038946),
 ('e', -0.0001899929693261363),
 ('áudio', -0.00018168594865986156),
 ('entrar', -0.00017688008298607667),
 ('Bolsonaro', 0.00017266254731474471),
 ('ajudar', -0.00015419628265536824),
 ('blindando', -0.00015033789100827948),
 ('Dilma', -0.00014768088696096886),
 ('salvando', 0.000141024012226897),
 ('o', -0.00012644132098685052),
